# ソートアルゴリズムをJavaScript実装の前に検証
### 性能テストと微調整


#### Quick Select + Merge-Insertion Sort

In [1]:
import numpy as np
import random


In [48]:
cnt = 0
def cmp_func(x,y):
    global cnt
    cnt += 1
    return (x>y)

def partition(arr, left, right, pivotIndex, cmp_func):
    pivotVal = arr[pivotIndex]
    arr[pivotIndex],arr[right] = arr[right], arr[pivotIndex]
    storeIndex = left
    for i in range(left, right):
        if cmp_func(arr[i], pivotVal):
            arr[storeIndex], arr[i] = arr[i], arr[storeIndex]
            storeIndex += 1
    arr[right], arr[storeIndex] = arr[storeIndex], arr[right]
    return storeIndex

def select(arr, k, left, right, cmp_func):
    pivotIndex = select_pivot_idx(arr, left, right, cmp_func)
    print("Select: ", arr[left:right+1], k, arr[pivotIndex])
    pivotNewIndex = partition(arr, left, right, pivotIndex, cmp_func)
    # print("Partition: ", arr[left:right+1], k, pivotNewIndex)
    if k==pivotNewIndex or k+1==pivotNewIndex:
        return arr[:k+1]
    elif k < pivotNewIndex: #パーティションの左側
        return select(arr, k, left, pivotNewIndex-1, cmp_func)
    else: # パーティションの右側
        return select(arr, k, pivotNewIndex+1, right, cmp_func)
    
def select_pivot_idx(arr, left, right, cmp_func):
    if right-left+1 < 3:
        return arr[0]
    else:
        x = arr[left:left+3]
        if cmp_func(x[0], x[1]):
            if cmp_func(x[1], x[2]):
                return left+1
            elif cmp_func(x[0], x[2]):
                return left+2
            else:
                return left
        else:
            if cmp_func(x[0], x[2]):
                return left
            elif cmp_func(x[1], x[2]):
                return left+2
            else:
                return left+1




In [57]:
N = 30
K = 10
cnt = 0
# arr =  [9, 2, 5, 7, 8, 6, 0, 3, 4, 1]
arr = list(range(N))
random.shuffle(arr)
print("Original arr:",arr)

print(arr, cnt)
# idx = partition(arr, 0, N-1, 0, cmp_func)
sub_arr = select(arr, K-1, 0, N-1, cmp_func)
print(arr, cnt)

Original arr: [15, 13, 24, 28, 10, 4, 1, 2, 8, 27, 26, 22, 29, 25, 6, 9, 7, 23, 3, 12, 11, 17, 0, 18, 21, 16, 20, 5, 19, 14]
[15, 13, 24, 28, 10, 4, 1, 2, 8, 27, 26, 22, 29, 25, 6, 9, 7, 23, 3, 12, 11, 17, 0, 18, 21, 16, 20, 5, 19, 14] 0
Select:  [15, 13, 24, 28, 10, 4, 1, 2, 8, 27, 26, 22, 29, 25, 6, 9, 7, 23, 3, 12, 11, 17, 0, 18, 21, 16, 20, 5, 19, 14] 9 15
Select:  [24, 28, 27, 26, 22, 29, 25, 23, 17, 18, 21, 16, 20, 19] 9 27
Select:  [26, 22, 24, 25, 23, 17, 18, 21, 16, 20, 19] 9 24
Select:  [22, 23, 17, 18, 21, 16, 20, 19] 9 22
Select:  [17, 18, 21, 16, 20, 19] 9 18
Select:  [19, 21, 20] 9 20
[28, 29, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 9, 7, 2, 3, 12, 11, 8, 0, 14, 13, 10, 4, 5, 1, 6] 83


In [64]:
# Implementation of the Merge Insertion sort algorithm
# -- a sort algorithm that is efficient w.r.t. number of pairwise comparisons
# based on the description in
# https://en.wikipedia.org/w/index.php?title=Merge-insertion_sort&oldid=959884881
# This is not intended to be time- or space-efficient.
# "less" is a function such that less(x,y) is true iff x should go to the left of y
# Dominik Peters, August 2020

# imports only needed for test at the bottom
from __future__ import print_function
import itertools

def binary_insert(seq, x, less):
    possible_positions = range(len(seq) + 1)
    L, R = 0, possible_positions[-1]
    while len(possible_positions) > 1:
        m = (L + R)//2
        if less(x, seq[m]):
            R = m
        else:
            L = m+1
        possible_positions = [p for p in possible_positions if L <= p <= R]
    return possible_positions[0]

def merge_insertion(seq, less):
    if len(seq) <= 1:
        return seq
    pairs = []
    for x1, x2 in zip(seq[::2], seq[1::2]):
        if less(x1, x2):
            pairs.append((x1, x2))
        else:
            pairs.append((x2, x1))
    # sort pairs by their large element
    pairs = merge_insertion(pairs, less=lambda a,b : less(a[1], b[1]))
    out = [x2 for x1, x2 in pairs]
    remaining = pairs[:]
    if len(seq) % 2 == 1:
        remaining.append((seq[-1], "END"))
    out.insert(0, remaining.pop(0)[0])
    # reorder remaining
    buckets = [2, 2]
    power_of_2 = 4
    while sum(buckets) < len(remaining):
        power_of_2 *= 2
        buckets.append(power_of_2 - buckets[-1])
    reordered = []
    last_index = 0
    for bucket in buckets:
        reordered += reversed(remaining[last_index:last_index+bucket])
        last_index += bucket
    for y, x in reordered:
        if x == "END":
            # insert unpaired element
            out.insert(binary_insert(out, y, less), y)
        else:
            # insert y in out by binary search up to but not including out.index(x)
            out.insert(binary_insert(out[:out.index(x)], y, less), y)
    return out

# test by calculating worst-case number of comparisons
counter = 0
def count(x,y):
    global counter
    counter += 1
    return x < y

N = 30
K = 10
cnt = 0
# arr =  [9, 2, 5, 7, 8, 6, 0, 3, 4, 1]
arr = list(range(N))
random.shuffle(arr)
print("Original arr:",arr)
merge_insertion(arr, count)
print(arr, counter)

# worsts = []
# for m in range(9):
#     worst = 0
#     for perm in itertools.permutations(range(m)):
#         counter = 0
#         assert list(merge_insertion(perm, less=count)) == list(range(m))
#         worst = max(worst, counter)
#     worsts.append(worst)
# print(worsts) # yields [0, 0, 1, 3, 5, 7, 10, 13, 16]

Original arr: [14, 9, 15, 12, 26, 3, 22, 8, 18, 10, 17, 24, 11, 19, 20, 29, 25, 7, 23, 13, 6, 5, 1, 0, 16, 28, 4, 21, 27, 2]
[14, 9, 15, 12, 26, 3, 22, 8, 18, 10, 17, 24, 11, 19, 20, 29, 25, 7, 23, 13, 6, 5, 1, 0, 16, 28, 4, 21, 27, 2] 108
